<a href="https://colab.research.google.com/github/Vac1k/Aurora_Project/blob/main/Aurora_Web_2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pyngrok gensim python-docx pandas numpy scikit-learn sentence-transformers

from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document
import gensim.downloader as api


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 839.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
file_path_inventory = "/content/Залишки по ТТ.xlsx"
inventory_df = pd.read_excel(file_path_inventory)

inventory_df = inventory_df.rename(columns={
    "Рівень категорії 3": "Category_Level3",
    "Назва товару": "Product_Full_Name",
    "К-сть залишків на кінець дня": "Stock_Quantity_End_Day",
    "Собівартість залишків на кінець дня": "Stock_Cost_End_Day"
})
inventory_df["Product_ID"] = pd.to_numeric(inventory_df["Product_Full_Name"].str.extract(r'^(\d+)')[0], errors='coerce')
inventory_df["Product_Name"] = inventory_df["Product_Full_Name"].str.replace(r'^\d+\s+', '', regex=True)
inventory_df["Calculated_Price"] = (inventory_df["Stock_Cost_End_Day"] / inventory_df["Stock_Quantity_End_Day"]).round(3)


In [ ]:
ai_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
inventory_df["Product_Vector"] = inventory_df["Product_Name"].apply(lambda x: ai_model.encode(x))
word2vec_model = api.load("word2vec-ruscorpora-300")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[==================================================] 100.0% 198.8/198.8MB downloaded


In [ ]:
def load_events_from_docx():
    doc = Document("/content/12123.docx")
    events_dict, current_event = {}, None
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        if not text:
            continue
        if text[0].isdigit() and "." in text:
            current_event = text.split(". ", 1)[1]
            events_dict[current_event] = []
        elif current_event:
            events_dict[current_event].append(text)
    return events_dict

events_data = load_events_from_docx()

In [ ]:
def find_similar_words(word, top_n=3):
    try:
        return [w[0] for w in word2vec_model.most_similar(word, topn=top_n)]
    except KeyError:
        return []

In [ ]:
def filter_by_event(event_name):
    allowed_keywords = events_data.get(event_name, [])
    if not allowed_keywords:
        return pd.DataFrame(columns=inventory_df.columns)

    filtered_df = inventory_df[inventory_df["Product_Name"].apply(
        lambda x: any(word.lower() in x.lower() for word in allowed_keywords)
    )]

    if filtered_df.empty:
        return find_similar_products(event_name, top_n=5)

    return filtered_df.head(5).to_dict(orient="records")

In [ ]:
def find_similar_products(query, top_n=5):
    query_vector = np.array(ai_model.encode(query)).reshape(1, -1)
    product_vectors = np.array(list(inventory_df["Product_Vector"]))

    similarity_scores = cosine_similarity(query_vector, product_vectors).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    return inventory_df.iloc[top_indices][["Product_Name", "Category_Level3", "Stock_Quantity_End_Day"]].head(top_n).to_dict(orient="records")


In [ ]:
def process_user_query(user_input):
    user_input = user_input.lower().strip()
    matched_event = next((event for event in events_data if event.lower() in user_input), None)

    if matched_event:
        return filter_by_event(matched_event)

    expanded_keywords = user_input.split()
    for word in user_input.split():
        expanded_keywords.extend(find_similar_words(word))

    return find_similar_products(" ".join(expanded_keywords), top_n=5)

In [ ]:
html_template = '''
<!DOCTYPE html>
<html lang="uk">
<head>
    <meta charset="UTF-8">
    <title>AI Рекомендації</title>
    <script>
        async function getRecommendations() {
            const userQuery = document.getElementById("query").value;
            const response = await fetch('/recommend', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({query: userQuery})
            });
            const data = await response.json();
            document.getElementById("results").innerHTML = data.map((item, i) =>
                `<p><strong>${i+1}.</strong> ${item.Product_Name} - ${item.Category_Level3} - Кількість: ${item.Stock_Quantity_End_Day}</p>`
            ).join('');
        }
    </script>
</head>
<body>
    <h2>🔍 Введіть ваш запит для отримання рекомендацій</h2>
    <input type="text" id="query" placeholder="Введіть запит...">
    <button onclick="getRecommendations()">Пошук</button>
    <div id="results"></div>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(html_template)

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.json
    query = data.get('query', '')
    recommendations = process_user_query(query)
    return jsonify(recommendations)

port = 5000
public_url = ngrok.connect(port)
print(f"🔗 Посилання для тестування: {public_url}")

app.run(port=port)

🔗 Посилання для тестування: NgrokTunnel: "https://dff9-35-198-197-96.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 14:55:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 14:55:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:00:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:00:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:00:23] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:00:49] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:00:56] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:01:29] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:01:38] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 15:01:41] "POST /recommend H

In [ ]:
app = Flask(__name__)
!ngrok config add-authtoken 2tPMHBHBtoCyuWXLK2cgkhcpNvl_3g3KYNbVTLmfjr4QgyvQ7

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
